In [1]:
import yfinance as yf
from langchain_core.tools import tool
from langchain_core.tools import render_text_description
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser


#### Define Tool

In [2]:
@tool
def get_stock_price(symbol: str) -> float:
    """
    Fetches the current stock price for a given ticker symbol.
    This function attempts to retrieve the stock price using various attributes
    from the Yahoo Finance API. It first checks for the price in the following
    attributes in order: "regularMarketPrice", "currentPrice", and "price".
    If none of these attributes contain a valid price, it then checks the
    "last_price" attribute in the fast_info object.
    Args:
        symbol (str): The ticker symbol of the stock.
    Returns:
        float: The current stock price.
    Raises:
        Exception: If no valid price data is found.
    """

    ticker = yf.Ticker(symbol)
    price_attrs = ["regularMarketPrice", "currentPrice", "price"]

    for attr in price_attrs:
        if attr in ticker.info and ticker.info[attr] is not None:
            return ticker.info[attr]

    fast_info = ticker.fast_info
    if hasattr(fast_info, "last_price") and fast_info.last_price is not None:
        return fast_info.last_price

    raise Exception("Could not find valid price data")

In [3]:
tools = [get_stock_price]

In [ ]:
rendered_tools = render_text_description(tools)
print(rendered_tools)

#### Prompt

In [ ]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)
prompt

#### Initialize Local Ollama Model (Deepseek R1 1.5b)

In [6]:
model = OllamaLLM(model="deepseek-r1:1.5b")

#### Setup tool to call by llm

In [7]:
class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

#### Create the chain and invoke the tool in runtime

In [8]:
chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)


In [10]:
chain.invoke({"input": "I wanna know the stock for Apple"})

{'name': 'get_stock_price',
 'arguments': {'symbol': 'AAPL'},
 'output': 220.6099}